# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [9]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [10]:
instrument_id = 'PHILIPS_A'

### Connect to Exchange

In [48]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2021-01-23 18:08:00,611 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-01-23 18:08:00,782 [client    ] [Thread-7    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


## Outstanding Orders, Trades, Current Positions and PnL

In [50]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [55]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [54]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [59]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

PHILIPS_A -450
PHILIPS_B 0


In [18]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

PHILIPS_A {'volume': 50, 'cash': -3298.4000000000015}
PHILIPS_B {'volume': 0, 'cash': -1207.1999999999994}


In [19]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

-635.6000000000004


## Order Book and Public Trade Ticks

In [1]:
book = e.get_last_price_book(instrument_id)
print(book)

NameError: name 'e' is not defined

In [70]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

[PHILIPS_B] price(64.4), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.5), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.5), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(20), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.60000000000001), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.7), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.5), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.5), volume(10), aggressor_side(bid), buyer(), seller()
[PHILIPS_B] price(64.4), volume(10), aggressor_side(ask), buyer(), seller()

In [71]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [23]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [34]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=73, volume=100, side='bid', order_type='limit')
print(f"Order Id: {result}")

AssertionError: Cannot call function until connected. Call connect() first

In [171]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=75, volume=100, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 208246


In [80]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=65, volume=10, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 181167


In [33]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=68, volume=100, side='ask', order_type='ioc')
print(f"Order Id: {result}")

2021-01-23 18:04:49,491 [client    ] [Thread-6    ] Forcing a disconnect due to an error: Pre-trade position limit (-500) breached on instrument PHILIPS_A. Old position=-450 New position=-550.


KjException: (remote):0: failed: remote exception: std::exception: Pre-trade position limit (-500) breached on instrument PHILIPS_A. Old position=-450 New position=-550
stack: 7f25633e3e80 7f25633e27b0 7f2563369666 7f2563358572 7f25633586a9 7f256334c6a4 7f25632a8069 7f25632af998 7f258d6e87b4 7f258d658cfe 7f25801d8e5a 7f25801da501 7f258d647f32 7f258d6e4aea 7f258d6ee0d9 7f258d6ee502 7f258d6e2e96 7f258d6ee0d9 7f258d6ee502 7f258d6e2e96 7f258d6ee0d9 7f258d6ee502 7f258d6e2e96 7f258d6ee0d9 7f258d6ee502 7f258d6e2e96 7f258d6ef691 7f258d6481ad 7f258d6482b1 7f258d647f32 7f258d6e4aea 7f258d6ee0d9

In [ ]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")

In [ ]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

In [ ]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

## 'Hack' Out of Positions

In [147]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'PHILIPS_A': 0, 'PHILIPS_B': 0}
{'PHILIPS_A': 0, 'PHILIPS_B': 0}
